<a href="https://colab.research.google.com/github/HyeJin816/ESAA_22/blob/main/0328_ch7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ch.7 앙상블 학습과 랜덤 포레스트**
#### 0328(월) 과제 : 핸즈온 245~271p

+ **앙상블 앙법** : 예측기(분류, 회귀 모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있는 학습 알고리즘.  
+ **랜덤 포레트스** : 각 개별 트리의 예측을 구하여 결정트리 분류기를 훈련시키고 가장 많은 선택을 받은 클래스를 예측으로 선택.  

-----  
## **7.1 투표 기반 분류기**


+ **직접투표 분류기** : 각 분류기의 예측을 모아서 다수결로 정해지는 분류기 선택
> 다수결 투표 분류기가 랜덤추측보다 더 높은 성능인 *'약한 학습기'*라해도 앙상볼은 높은 정확도의 *'강한 학습기'*가 될 수 있음.  
+ **큰 수의 법칙** : 반복 횟수가 많아질 수록 확률이 증가  
> 적당한 정확도의 분류기로 많은 클래스 예측을 통해 높은 정확도 기대  
> 같은 데이터로 훈련시키기 때문에 같은 종류의 오차를 생성해 잘못된 클래스가 발생하고 앙상블의 정확도 저하

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [4]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.912


+ **간접투표** : 모든 분류기가 클래스의 확률을 예측할 수 있으면 전체 예측의 평균으로 가장 높은 클래스 예측 가능. 
> 확률이 높은 투표에 비중을 두어 직접 투표보다 높은 성능  
> ```voting='hard```가 아닌 ```voting='soft'```지정  
> ```probability=True```로 지정하여 클래스 확률 추정을 위한 교차 검증 진행

-----  
## **7.2 배깅과 페이스팅**


+ 분류기 생성 - 훈련세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시킴.  
> **배깅** : 훈련 세트에서 중복을 허용하여 샘플링  
> **페이스팅** : 훈련 세트에서 중복을 허용하지 않고 샘플링

### **7.2.1. 사이킷런의 배깅과 페이스팅**

In [5]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)

bag_clf.fit(X_train, y_train)

y_pred = bag_clf.predict(X_test)

### **7.2.2. oob 평가**  

**BaggingClassifier** : 기본값을 중복으로 허용하여 (```bootstrap=True```) 훈련 세트의 크기만큼 샘플링하는데, 이때 선택받지 못한 샘플을 **oob 샘플**이라고 함.  
> 예측기 훈련 동안은 별도 검정 세트가 아닌 oob 샘플로 평가  
> ```oob_score=True``` 지정하여 oob 평가 수행 확인

In [8]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)

bag_clf.oob_score_

0.9013333333333333

In [10]:
# 확인
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.896

-----  
## **7.3 랜덤 패치와 랜덤 서브 스페이스**


+ **랜덤 패치 방식** : 특성에 대한 샘플링에서 훈련 특성과 샘플을 모두 샘플링  
+ **랜덤 서브스페이스 방식** : 훈련 샘플을 모두 사용하면서 특성은 샘플링(```bootstrap=False```, ```bootstrap_features=True```)

-----  
## **7.4 랜덤 포레스트**


+ **랜덤 포레스트** : 일반적 배깅/페이스팅 방법을 적용한 결정 트리 앙상블  
> ```max_sample```로 훈련 세트 크기 지정  
> ```RandomForestClassifier``` 분류기 사용이 편리

In [11]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [12]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features='auto', max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

### **7.4.1. 엑스트라 트리**

+ **익스트림 랜덤 트리** (**엑스트라 트리**) : 무작위한 트리를 만들기 위해 후보 특성을 무작위로 분할하여 최상의 분할을 선택한 트리의 랜덤 포레스트  
> 편향이 크지만 분산을 낮춤  
> 모든 노드마다 최적의 임계값을 찾아야 해서 많은 시간 소요  
> ```ExtraTreesClassifier```

### **7.4.2. 특성 중요도**

> 어떤 특성을 사용한 노드가 평균적으로 분순도를 얼마나 감소시키는지에 따라 특성의 중요도 측정 (가중치 평균)





In [14]:
from sklearn.datasets import load_iris
iris = load_iris()

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])

for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09817584087035154
sepal width (cm) 0.024100857331023757
petal length (cm) 0.45126865721826603
petal width (cm) 0.4264546445803587


-----  
## **7.5 부스팅**


+ **부스팅** : 약한 학습기를 여러개 연결하여 강한 학습기를 만드는 앙상블  
> 에이다부스트 / 그레이디언트 부스팅

### **7.5.1. 에이다부스트**

* 이전 모델이 과소적합했던 훈련 샘플의 가중치를 높여서 어려운 샘플에 점점 맞춰지게 되는 것.  
> 기존 분류기 알고리즘에서 잘못 분류된 샘플의 가중치를 높이고 계속해서 업데이트  
> 가중치가 적용된 훈련 세트의 전반적인 정확도에 따라서 다른 가중치 적용  
> ```AdaBoostClassifier``` 얕은 결정트리 기반( ```max_depth=1``` )

In [16]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm='SAMME.R', learning_rate=0.5)

ada_clf.fit(X_train,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### **7.5.2. 그레이디언트 부스팅**

+ **그레이디언트 트리 부스팅** (**그레이디언트 부스티드 회귀 트리**) : 이전의 오차를 보정하도록 예측기를 순차적으로 추가. 샘플의 가중치 대신 이전 예측기의 잔여 오차로 학습  

In [17]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [18]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [19]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [21]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=87)

In [23]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break

In [26]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

[01:25:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [27]:
xgb_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[01:25:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.469
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.439454
[2]	validation_0-rmse:0.414102
[3]	validation_0-rmse:0.391749
[4]	validation_0-rmse:0.37372
[5]	validation_0-rmse:0.356407
[6]	validation_0-rmse:0.342924
[7]	validation_0-rmse:0.32897
[8]	validation_0-rmse:0.318788
[9]	validation_0-rmse:0.309996
[10]	validation_0-rmse:0.300987
[11]	validation_0-rmse:0.294586
[12]	validation_0-rmse:0.287137
[13]	validation_0-rmse:0.280298
[14]	validation_0-rmse:0.275012
[15]	validation_0-rmse:0.270456
[16]	validation_0-rmse:0.266698
[17]	validation_0-rmse:0.262019
[18]	validation_0-rmse:0.259213
[19]	validation_0-rmse:0.255453
[20]	validation_0-rmse:0.252828
[21]	validation_0-rmse:0.250606
[22]	validation_0-rmse:0.24843
[23]	validation_0-rmse:0.246899
[24]	validation_0-rmse:0.245072
[25]	validation_0-rmse:0.

-----  
## **7.6 스태킹**


+ **스태킹** : 앙상블에 속한 모든 예측기의 예측을 취합하여 최종 예측 생성